In [460]:
from qiskit.optimization import QuadraticProgram
import time
from typing import List, Optional, Any
import numpy as np
import matplotlib.pyplot as plt

from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer
from qiskit.optimization.algorithms import CplexOptimizer
backend=BasicAer.get_backend('qasm_simulator')

In [529]:
#Step 2: Create the QP from the Docplex model
filename='301-Assignment Problem-MILP-B009-133066'
mod = QuadraticProgram()
mod.read_from_lp_file('..\problemFiles\\'+filename+'.lp')
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Assignment Problem

Maximize
 obj: 90 x_0_0 + 80 x_0_1 + 75 x_0_2 + 35 x_1_0 + 85 x_1_1 + 55 x_1_2 + 45 x_2_0
      + 110 x_2_1 + 95 x_2_2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 <= 1
 c2: x_1_0 + x_1_1 + x_1_2 <= 1
 c3: x_2_0 + x_2_1 + x_2_2 <= 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



In [530]:
#Access the constant, the linear term, and the quadratic term from Quadratic.objective
print('constant:\t\t\t', mod.objective.constant)
print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())

#As for linear and quadratic terms, we get a dense matrix (to_array), a sparse matrix (coefficients), 
#and a dictionary (to_dict). 
#For dictionaries, we may specify whether to use variable indices or names as keys. 
#Note that the quadratic terms are stored in a compressed way, 
#e.g., {('x', 'y'): 1, ('y', 'x'): 2} is stored as {('x', 'y'): 3}. 
#You can get the quadratic term as a symmetric matrix by calling to_array(symmetric=True) or 
#to_dict(symmetric=True). 
#If you call to_dict(name=True), you can get a dictionary whose keys are pairs of variable names.
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')
print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)

constant:			 0.0
linear dict:			 {0: 90.0, 1: 80.0, 2: 75.0, 3: 35.0, 4: 85.0, 5: 55.0, 6: 45.0, 7: 110.0, 8: 95.0}
linear array:			 [ 90.  80.  75.  35.  85.  55.  45. 110.  95.]
linear array as sparse matrix:
   (0, 0)	90.0
  (0, 1)	80.0
  (0, 2)	75.0
  (0, 3)	35.0
  (0, 4)	85.0
  (0, 5)	55.0
  (0, 6)	45.0
  (0, 7)	110.0
  (0, 8)	95.0 

quadratic dict w/ index:	 {}
quadratic dict w/ name:		 {}
symmetric quadratic dict w/ name:	 {}
quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

symmetric quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [

In [531]:
# define COBYLA optimizer to handle convex continuous problems.
cobyla = CobylaOptimizer()

# define QAOA via the minimum eigen optimizer
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=backend))

# exact QUBO solver as classical benchmark
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs
cplex = CplexOptimizer()

In [532]:
qp=mod

In [533]:
admm_params = ADMMParameters(
                            #rho_initial=1001,
                            #beta=1000,
                            #factor_c=900,
                            maxiter=100
                            #three_block=True, tol=1.e-6
                        )

In [534]:
# define QUBO optimizer
#qubo_optimizer = exact
qubo_optimizer = cplex  # uncomment to use CPLEX instead

# define classical optimizer
convex_optimizer = cobyla
#convex_optimizer = cplex  # uncomment to use CPLEX instead

# initialize ADMM with classical QUBO and convex optimizer
admm = ADMMOptimizer(params=admm_params,
                     qubo_optimizer=qubo_optimizer,
                     continuous_optimizer=convex_optimizer)

In [535]:
# run ADMM to solve problem
import logging
from qiskit.aqua import set_qiskit_aqua_logging
set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL
#set_qiskit_aqua_logging(logging.NOTSET)  # OFF

startTime=time.time()
result = admm.solve(qp)
endTime = time.time()

In [536]:
#set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL
set_qiskit_aqua_logging(logging.NOTSET)  # OFF
#print("fval={:.2f}".format(result.fval))
def log_results(env='Classical'):
    #pe	variables	Optimal Function Value	Result	RunTime (seconds)	Total Quantum RunTime (second)	Qubits	Operations	Environment	Solver	Solution
    entry=filename+','+'MILP'+','+str(qp.get_num_vars())+','+'%.4f'%result.fval+','
    for x in result.x:
        entry = entry+'%.4f'%(x)+'  '
    
    solver=qubo_optimizer.__class__.__name__
    qubits=0
    operations=0
    if env!='Classical':
        solver=solver+'('+qubo_optimizer.min_eigen_solver.__class__.__name__+')'
        qubits = qubo_optimizer.min_eigen_solver.get_optimal_circuit().num_qubits
        operations=qubo_optimizer.min_eigen_solver.get_optimal_circuit().size()
        
    entry=entry+','+'%.2f'%(endTime-startTime)+',,'+str(qubits)+','+str(operations)+','+env+','+ \
        'ADMM with '+solver+' and '+convex_optimizer.__class__.__name__+','+str(result.status).split('.')[1]+'\n'
    #print(entry)
    f=open('..\\stats.csv','a')
    f.write(entry)
    f.close()

log_results()
print(result)

optimal function value: 295.0
optimal value: [1. 0. 0. 0. 0. 0. 0. 1. 1.]
status: INFEASIBLE


In [537]:
# define QUBO optimizer
qubo_optimizer = qaoa

# define classical optimizer
convex_optimizer = cobyla
# convex_optimizer = cplex  # uncomment to use CPLEX instead

# initialize ADMM with quantum QUBO optimizer and classical convex optimizer
admm_q = ADMMOptimizer(#params=admm_params,
                       qubo_optimizer=qubo_optimizer,
                       continuous_optimizer=convex_optimizer)



In [538]:
set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL
#set_qiskit_aqua_logging(logging.NOTSET)  # OFF

startTime=time.time()
result_q = admm_q.solve(qp)
endTime = time.time()

2021-01-01 20:11:52,493:qiskit.aqua.algorithms.vq_algorithm:INFO: Starting optimizer.
bounds=[(0, 3.141592653589793), (0, 6.283185307179586)]
initial point=[0, 0]
2021-01-01 20:11:52,725:qiskit.aqua.quantum_instance:DEBUG: ==== Before transpiler ====
2021-01-01 20:11:52,725:qiskit.aqua.quantum_instance:DEBUG: Submitting 1 circuits.
0-th circuit: 9 qubits, 9 classical bits and 63 operations with depth 13
op_counts: OrderedDict([('cx', 18), ('rz', 18), ('h', 9), ('rx', 9), ('measure', 9)])

2021-01-01 20:11:52,725:qiskit.aqua.quantum_instance:DEBUG: ====  After transpiler ====
2021-01-01 20:11:52,725:qiskit.aqua.quantum_instance:DEBUG: Submitting 1 circuits.
0-th circuit: 9 qubits, 9 classical bits and 63 operations with depth 13
op_counts: OrderedDict([('cx', 18), ('u1', 18), ('u2', 9), ('u3', 9), ('measure', 9)])

2021-01-01 20:11:52,725:qiskit.aqua.operators.converters.circuit_sampler:DEBUG: Parameter binding 0.00000 (ms)
2021-01-01 20:11:53,311:qiskit.aqua.algorithms.minimum_eigen_so

In [539]:
#set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL
set_qiskit_aqua_logging(logging.NOTSET)  # OFF
log_results(env=backend.name())
print(result)
#print("fval={:.2f}".format(result.fval))
#print('Time : ', endTime-startTime)

optimal function value: 295.0
optimal value: [1. 0. 0. 0. 0. 0. 0. 1. 1.]
status: INFEASIBLE


In [540]:
print(qaoa.min_eigen_solver.get_optimal_circuit().draw())

     ┌───┐                                                 ┌───┐┌───────┐     »
q_0: ┤ H ├─────────────────────────────────────────────────┤ X ├┤ RZ(0) ├─────»
     ├───┤                                            ┌───┐└─┬─┘├───────┤     »
q_1: ┤ H ├────────────────────────────────────────────┤ X ├──┼──┤ RZ(0) ├─────»
     ├───┤                                       ┌───┐└─┬─┘  │  ├───────┤┌───┐»
q_2: ┤ H ├───────────────────────────────────────┤ X ├──┼────┼──┤ RZ(0) ├┤ X ├»
     ├───┤          ┌───┐┌───────┐          ┌───┐└─┬─┘  │    │  └───────┘└─┬─┘»
q_3: ┤ H ├──────────┤ X ├┤ RZ(0) ├──────────┤ X ├──┼────┼────┼─────────────┼──»
     ├───┤     ┌───┐└─┬─┘├───────┤     ┌───┐└─┬─┘  │    │    │             │  »
q_4: ┤ H ├─────┤ X ├──┼──┤ RZ(0) ├─────┤ X ├──┼────┼────┼────┼─────────────┼──»
     ├───┤┌───┐└─┬─┘  │  ├───────┤┌───┐└─┬─┘  │    │    │    │             │  »
q_5: ┤ H ├┤ X ├──┼────┼──┤ RZ(0) ├┤ X ├──┼────┼────┼────┼────┼─────────────┼──»
     ├───┤└─┬─┘  │    │  └───────┘└─┬─┘ 